In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Understanding Bayesian Optimization

[Bayesian optimization](https://arxiv.org/abs/1012.2599) is a powerful strategy for **finding the extrema of objective functions** that are expensive to evaluate. It is particularly useful when these evaluations are costly, when one does not have access to derivatives, or when the problem is non-convex.

- **Objective function**

- **Surrogate function**: Bayesian approximation of the objective function that can be sampled efficiently.

- **Acquisition function**: Technique by which the posterior is used to select the next sample from the search space.

The **Bayesian Optimization algorithm** can be summarized as follows.

- 1. Select a sample by optimizing the Acquisition function.

- 2. Evaluate the sample with the Objective function.

- 3. Update data and, in turn, the Surrogate function.

- 4. Go to 1.

URL: https://machinelearningmastery.com/what-is-bayesian-optimization/

In [ ]:
import numpy as np
import pandas as pd
from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot

## Objective function

In [ ]:
# ideal multimodal function
def multimodal(x):
    return (x**2 * sin(5 * pi * x)**6.0)

In [ ]:
# objective function
def objective(x, noise=0.1):
    noise = normal(loc=0, scale=noise)
    return multimodal(x) + noise

## Input data

In [ ]:
# Input
X = random(100)
y = asarray([objective(x) for x in X])

# find optima
ix = argmax(y)
print('Optima: x=%.3f, y=%.3f' % (X[ix], y[ix]))

## Approximation for the objective function

In [ ]:
# reshape into rows and cols
X = X.reshape(len(X), 1)
y = y.reshape(len(y), 1)

# define and fit the model
model = GaussianProcessRegressor()
model.fit(X, y)

# approximation for the objective function by surrogate function
def surrogate(model, X):
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")      
        return model.predict(X, return_std=True)

### Vizualization

In [ ]:
# plot real observations - multimodal function - surrogate function
def plot(X, y, model):   

    # scatter plot of inputs and real objective function
    pyplot.scatter(X, y, color='g', marker='.', s=20)

    # line plot of multimodal function
    X_mm = asarray(arange(0, 1, 0.001))
    y_mm = asarray([multimodal(x) for x in X_mm])
    pyplot.plot(X_mm, y_mm, linestyle='dashed', linewidth=0.5)
    
    # line plot of surrogate function across domain
    X_model = X_mm.reshape(len(X_mm), 1)
    y_model, _ = surrogate(model, X_model)
    pyplot.plot(X_model, y_model)
    
# plot the surrogate function
plot(X, y, model)

## Perform the optimization process

### Acquisition function and its optimization

In [ ]:
# probability of improvement acquisition function
def acquisition(X, Xsamples, model):

    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)

    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    mu = mu[0]   # mu = mu[:, 0]
    
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs
 

# optimize the acquisition function
def opt_acquisition(X, y, model):

    # random search, generate random samples
    Xsamples = random(100)
    Xsamples = Xsamples.reshape(len(Xsamples), 1)

    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)

    # locate the index of the largest scores
    ix = argmax(scores)
    
    return Xsamples[ix, 0]

### Optimization process

In [ ]:
# Perform the optimization process
for i in range(500):
    # select the next point to sample
    x = opt_acquisition(X, y, model)

    # sample the point
    actual = objective(x)

    # summarize the finding
    est, _ = surrogate(model, [[x]])
#     print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))

    # add the data to the dataset
    X = vstack((X, [[x]]))
    y = vstack((y, [[actual]]))
    
    # update the model
    model.fit(X, y)

### Vizualization and the best result

In [ ]:
# plot all samples and the final surrogate function
plot(X, y, model)
    
# best result
ix = argmax(y)
print('Best Result: x=%.3f, y=%.3f' % (X[ix], y[ix]))